In [1]:
import warnings
warnings.filterwarnings(action="ignore")
import datetime
import time
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import optuna
from optuna.samplers import TPESampler
from optuna.pruners import HyperbandPruner

In [3]:
RANDOM_SEED = 99

In [2]:
train_df = pd.read_parquet('./data/train_v2_1_nn.parquet')
train_null_df = pd.read_parquet('./data/train_v2_1_n.parquet').drop(columns=["AIR_TEMPERATURE"])
test_df = pd.read_parquet('./data/test_v2_1.parquet').drop(columns=["AIR_TEMPERATURE"])

In [5]:
train_df.head()

,AIR_TEMPERATURE,ARI_CO_BR,ARI_CO_CA,ARI_CO_CL,ARI_CO_CN,ARI_CO_FI,ARI_CO_ID,ARI_CO_IN,ARI_CO_JP,ARI_CO_KR,...,SHIP_TYPE_CATEGORY_Container,SHIP_TYPE_CATEGORY_Tanker,WITlt0,WTI,day,hour,minute,month,weekday,year
0,15.9,0,0,0,1,0,0,0,0,0,...,0,0,0,-1.081668,15,4,3,10,3,2020
1,24.5,0,0,0,1,0,0,0,0,0,...,1,0,0,-0.188614,17,2,55,9,1,2019
2,9.4,0,0,0,1,0,0,0,0,0,...,1,0,0,-0.305226,23,6,43,2,5,2019
3,22.1,0,0,0,0,0,0,0,1,0,...,0,0,0,-1.074380,18,22,6,9,4,2020
4,22.8,0,0,0,0,0,0,0,0,0,...,1,0,0,1.209272,13,12,57,8,5,2022


In [7]:
train_null_df.head()

,ARI_CO_BR,ARI_CO_CA,ARI_CO_CL,ARI_CO_CN,ARI_CO_FI,ARI_CO_ID,ARI_CO_IN,ARI_CO_JP,ARI_CO_KR,ARI_CO_LV,...,SHIP_TYPE_CATEGORY_Container,SHIP_TYPE_CATEGORY_Tanker,WITlt0,WTI,day,hour,minute,month,weekday,year
5,0,0,0,1,0,0,0,0,0,0,...,1,0,0,-0.840670,8,14,24,9,1,2015
8,0,0,0,0,0,0,0,1,0,0,...,1,0,0,-0.488404,27,6,23,1,4,2017
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0.008168,3,7,8,2,5,2018
10,0,0,0,1,0,0,0,0,0,0,...,1,0,0,0.086881,14,3,33,8,1,2018
11,0,0,0,0,0,0,0,0,0,0,...,0,0,0,-1.714288,8,22,51,2,0,2016


In [6]:
test_df.head()

,ARI_CO_BR,ARI_CO_CA,ARI_CO_CL,ARI_CO_CN,ARI_CO_FI,ARI_CO_ID,ARI_CO_IN,ARI_CO_JP,ARI_CO_KR,ARI_CO_LV,...,SHIP_TYPE_CATEGORY_Container,SHIP_TYPE_CATEGORY_Tanker,WITlt0,WTI,day,hour,minute,month,weekday,year
0,0,0,0,0,0,0,0,1,0,0,...,0,1,0,1.279239,27,8,29,8,5,2022
1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1.816140,27,20,57,3,6,2022
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0.789955,18,2,6,1,2,2023
3,0,0,0,0,0,0,0,1,0,0,...,0,0,0,-1.152121,2,0,59,8,1,2016
4,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0.821537,24,0,27,1,1,2023


In [8]:
train_x = train_df.drop(columns=["AIR_TEMPERATURE"])
train_y = train_df["AIR_TEMPERATURE"]

In [9]:
def create_model(param):
    return xgb.XGBRegressor(
        random_state=RANDOM_SEED,
        tree_method="gpu_hist",
        eval_metric="mae",
        early_stopping_rounds=50,
        **param
    )

In [13]:
def train_model(param, X, y, X_test, trial=None, test=False):
  skf = KFold(n_splits=3, shuffle=True, random_state=RANDOM_SEED)

  val_scores = []
  y_tests = []
  models = []

  for idx, (train_idx, val_idx) in enumerate(skf.split(X, y)):

    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
    X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]

    model = create_model(param)
    model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_val, y_val)], verbose=1000)

    y_hat_val = model.predict(X_val)
    score = mean_squared_error(y_val.values, y_hat_val)
    val_scores.append(score)
    print(f'Fold: {idx+1}/3 score = {score:.5f}')

    if test:
      y_tests.append(model.predict(X_test))
      models.append(model)
    
    if trial:
      trial.report(score, idx)

      if trial.should_prune():
        raise optuna.TrialPruned()

  return val_scores, y_tests, models

In [14]:
def objective_xgb(trial):
  param = {
    "booster": trial.suggest_categorical("booster", ["gbtree", "dart"]),
    "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
    "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
    "subsample": trial.suggest_float("subsample", 0.1, 1.0),
    "colsample_bytree": trial.suggest_float("colsample_bytree", 0.1, 1.0),
    "n_estimators": trial.suggest_int("n_estimators", 100, 1000, step=100),
    "max_depth": trial.suggest_int("max_depth", 3, 9),
    "min_child_weight": trial.suggest_int("min_child_weight", 2, 10),
    "eta": trial.suggest_float("eta", 1e-8, 1.0, log=True),
    "gamma": trial.suggest_float("gamma", 1e-8, 1.0, log=True),
    "grow_policy": trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])
  }

  if param["booster"] == "dart":
    param['sample_type'] = trial.suggest_categorical('sample_type', ['uniform', 'weighted'])
    param['normalize_type'] = trial.suggest_categorical('normalize_type', ['tree', 'forest'])
    param['rate_drop'] = trial.suggest_float('rate_drop', 1e-8, 1.0, log=True)
    param['skip_drop'] = trial.suggest_float('skip_drop', 1e-8, 1.0, log=True)

  val_scores, y_tests, models = train_model(param, train_x, train_y, train_null_df, trial)

  return sum(val_scores) / len(val_scores)

In [15]:
study = optuna.create_study(
    sampler=TPESampler(seed=RANDOM_SEED),
    direction='minimize',
    study_name='xgb_tuning',
    pruner=HyperbandPruner(
      min_resource=1, max_resource=8, reduction_factor=3
    ),
)

study.optimize(objective_xgb, n_trials=50)

[I 2023-10-08 20:47:09,743] A new study created in memory with name: xgb_tuning


[0]	validation_0-mae:18.81049	validation_1-mae:18.72838
[299]	validation_0-mae:18.81043	validation_1-mae:18.72833
Fold: 1/3 score = 430.34211
[0]	validation_0-mae:18.76841	validation_1-mae:18.81253
[299]	validation_0-mae:18.76836	validation_1-mae:18.81248
Fold: 2/3 score = 432.90070
[0]	validation_0-mae:18.77046	validation_1-mae:18.80844
[299]	validation_0-mae:18.77040	validation_1-mae:18.80838


[I 2023-10-08 20:47:21,859] Trial 0 finished with value: 431.9718954260384 and parameters: {'booster': 'gbtree', 'lambda': 0.04017550942202233, 'alpha': 1.7847241527490656e-08, 'subsample': 0.8272449670283629, 'colsample_bytree': 0.6090556776494775, 'n_estimators': 300, 'max_depth': 3, 'min_child_weight': 10, 'eta': 1.1339812255777714e-08, 'gamma': 0.014399394343769443, 'grow_policy': 'depthwise'}. Best is trial 0 with value: 431.9718954260384.


Fold: 3/3 score = 432.67287
[0]	validation_0-mae:18.81045	validation_1-mae:18.72835
[899]	validation_0-mae:18.77740	validation_1-mae:18.69535
Fold: 1/3 score = 428.95237
[0]	validation_0-mae:18.76838	validation_1-mae:18.81250
[899]	validation_0-mae:18.73541	validation_1-mae:18.77954
Fold: 2/3 score = 431.50935
[0]	validation_0-mae:18.77042	validation_1-mae:18.80840
[899]	validation_0-mae:18.73746	validation_1-mae:18.77539


[I 2023-10-08 21:20:27,369] Trial 1 finished with value: 430.58085712661483 and parameters: {'booster': 'dart', 'lambda': 1.4575798463318516e-05, 'alpha': 0.6189425995687715, 'subsample': 0.5719732438377161, 'colsample_bytree': 0.1842517839679773, 'n_estimators': 900, 'max_depth': 4, 'min_child_weight': 6, 'eta': 2.178476742104051e-06, 'gamma': 0.033817296080697445, 'grow_policy': 'depthwise', 'sample_type': 'uniform', 'normalize_type': 'tree', 'rate_drop': 1.4193450623251104e-07, 'skip_drop': 4.983900949448835e-07}. Best is trial 1 with value: 430.58085712661483.


...

Fold: 3/3 score = 216.87074
[0]	validation_0-mae:18.74611	validation_1-mae:18.66424
[899]	validation_0-mae:2.79912	validation_1-mae:2.84490
Fold: 1/3 score = 14.23382
[0]	validation_0-mae:18.70411	validation_1-mae:18.74819
[899]	validation_0-mae:2.79309	validation_1-mae:2.86954
Fold: 2/3 score = 14.44703
[0]	validation_0-mae:18.70603	validation_1-mae:18.74390
[899]	validation_0-mae:2.79531	validation_1-mae:2.86247


[I 2023-10-09 02:16:11,960] Trial 48 finished with value: 14.376824903202321 and parameters: {'booster': 'dart', 'lambda': 0.014454633963208929, 'alpha': 0.8860111093915045, 'subsample': 0.5699838097442007, 'colsample_bytree': 0.8912614777575536, 'n_estimators': 900, 'max_depth': 9, 'min_child_weight': 2, 'eta': 0.0035555220270102952, 'gamma': 0.39886007158279546, 'grow_policy': 'depthwise', 'sample_type': 'weighted', 'normalize_type': 'tree', 'rate_drop': 0.001387395227365699, 'skip_drop': 5.938280298581349e-07}. Best is trial 45 with value: 6.551550399391943.


Fold: 3/3 score = 14.44962
[0]	validation_0-mae:18.80897	validation_1-mae:18.72687
[999]	validation_0-mae:17.36849	validation_1-mae:17.29173
Fold: 1/3 score = 368.35473
[0]	validation_0-mae:18.76690	validation_1-mae:18.81102
[999]	validation_0-mae:17.32997	validation_1-mae:17.37349


[I 2023-10-09 02:42:51,684] Trial 49 pruned. 


Fold: 2/3 score = 370.77628


In [16]:
trial = study.best_trial

In [17]:
print(trial.params)

{'booster': 'dart', 'lambda': 0.00965012888749584, 'alpha': 0.9842762221618983, 'subsample': 0.6858379830298229, 'colsample_bytree': 0.8913163153161074, 'n_estimators': 1000, 'max_depth': 9, 'min_child_weight': 3, 'eta': 0.03792582206301568, 'gamma': 0.2443561878178881, 'grow_policy': 'depthwise', 'sample_type': 'weighted', 'normalize_type': 'tree', 'rate_drop': 9.776329107921942e-05, 'skip_drop': 4.429240963896771e-06}


In [23]:
best_param = {
  'booster': 'dart',
  'lambda': 0.00965012888749584,
  'alpha': 0.9842762221618983,
  'subsample': 0.6858379830298229,
  'colsample_bytree': 0.8913163153161074,
  'n_estimators': 1000,
  'max_depth': 9,
  'min_child_weight': 3,
  'eta': 0.03792582206301568,
  'gamma': 0.2443561878178881,
  'grow_policy': 'depthwise',
  'sample_type': 'weighted',
  'normalize_type': 'tree',
  'rate_drop': 9.776329107921942e-05,
  'skip_drop': 4.429240963896771e-06
}

val_scores, y_tests, models = train_model(best_param, train_x, train_y, train_null_df, test=True)

[0]	validation_0-mae:18.12413	validation_1-mae:18.04451
[999]	validation_0-mae:1.32980	validation_1-mae:1.74364
Fold: 1/3 score = 6.50055
[0]	validation_0-mae:18.08292	validation_1-mae:18.12676
[999]	validation_0-mae:1.33361	validation_1-mae:1.76496
Fold: 2/3 score = 6.57057
[0]	validation_0-mae:18.08330	validation_1-mae:18.12001
[999]	validation_0-mae:1.32763	validation_1-mae:1.75673
Fold: 3/3 score = 6.58353


In [25]:
# save model
for idx, model in enumerate(models):
  file_name = f"./models/temp_model_{idx}.model"
  model.save_model(file_name)

In [27]:
train_null_df["AIR_TEMPERATURE"] = np.mean(y_tests, axis=0)

In [28]:
train_null_df.head()

,ARI_CO_BR,ARI_CO_CA,ARI_CO_CL,ARI_CO_CN,ARI_CO_FI,ARI_CO_ID,ARI_CO_IN,ARI_CO_JP,ARI_CO_KR,ARI_CO_LV,...,SHIP_TYPE_CATEGORY_Tanker,WITlt0,WTI,day,hour,minute,month,weekday,year,AIR_TEMPERATURE
5,0,0,0,1,0,0,0,0,0,0,...,0,0,-0.840670,8,14,24,9,1,2015,26.112450
8,0,0,0,0,0,0,0,1,0,0,...,0,0,-0.488404,27,6,23,1,4,2017,11.093024
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0.008168,3,7,8,2,5,2018,29.269526
10,0,0,0,1,0,0,0,0,0,0,...,0,0,0.086881,14,3,33,8,1,2018,25.843529
11,0,0,0,0,0,0,0,0,0,0,...,0,0,-1.714288,8,22,51,2,0,2016,29.435951


In [29]:
assert sorted(train_df.columns) == sorted(train_null_df.columns)

In [30]:
result = pd.concat([train_df, train_null_df], axis=0).sort_index()

In [31]:
result.head(10)

,AIR_TEMPERATURE,ARI_CO_BR,ARI_CO_CA,ARI_CO_CL,ARI_CO_CN,ARI_CO_FI,ARI_CO_ID,ARI_CO_IN,ARI_CO_JP,ARI_CO_KR,...,SHIP_TYPE_CATEGORY_Container,SHIP_TYPE_CATEGORY_Tanker,WITlt0,WTI,day,hour,minute,month,weekday,year
0,15.900000,0,0,0,1,0,0,0,0,0,...,0,0,0,-1.081668,15,4,3,10,3,2020
1,24.500000,0,0,0,1,0,0,0,0,0,...,1,0,0,-0.188614,17,2,55,9,1,2019
2,9.400000,0,0,0,1,0,0,0,0,0,...,1,0,0,-0.305226,23,6,43,2,5,2019
3,22.100000,0,0,0,0,0,0,0,1,0,...,0,0,0,-1.074380,18,22,6,9,4,2020
4,22.800000,0,0,0,0,0,0,0,0,0,...,1,0,0,1.209272,13,12,57,8,5,2022
5,26.112450,0,0,0,1,0,0,0,0,0,...,1,0,0,-0.840670,8,14,24,9,1,2015
6,4.400000,0,0,0,0,0,0,0,1,0,...,1,0,0,-0.507840,25,21,34,1,0,2021
7,21.800000,0,0,0,0,0,0,0,1,0,...,0,0,0,-0.459737,18,3,48,6,1,2019
8,11.093024,0,0,0,0,0,0,0,1,0,...,1,0,0,-0.488404,27,6,23,1,4,2017
9,29.269526,0,0,0,0,0,0,0,0,0,...,0,0,0,0.008168,3,7,8,2,5,2018


In [32]:
result.to_parquet("./data/train_v2_1.parquet")

In [33]:
test_temps = []
for model in models:
  test_temps.append(model.predict(test_df))

In [34]:
test_df["AIR_TEMPERATURE"] = np.mean(test_temps, axis=0)

In [35]:
assert sorted(train_df.columns) == sorted(test_df.columns)

In [36]:
test_df.to_parquet("./data/test_v2_1.parquet")